# **DATA UNDERSTANDING**

## **Context**

The Human Resources Analytics focus is to understand why the best and most experienced employees in the company. By the exploration of this dataset its possible to extract good insights of a problems that the Human Resource department deals daily. In many industries retain their best employees its a question of long term strategy, and can impact the companies growth or put in financial risk, mainly if the employees leave to work at the competitor.

## **Database Information**

Database yang dimiliki mempunyai 8 tabel, yaitu:
- Customers     : Menyimpan informasi tentang data pelanggan/customer.
- ```Employees```
    - First_Name : Nama Awal Employee
    - Last_Name : Nama Akhir Employee
    - Salary    : Pendapatan Employee
- ```Departments```
    - Department_Name   : Nama Departemen
- ```Job```
    - Job_Title : Jabatan/ Posisi di Perushaan
- locations
    - State_Province : Nama Provinsi asal Employee
- ```Countries```
    - Country_Name : Nama negara asal Employee

## QUESTION
    1. Proporsi Employee di Tiap Departemen
    2. Departemen manakah yang memiliki salary paling tinggi
    3. Job title apa saja yang mendapatkan commission?
    4. Dari negara manakah Employee paling banyak berasal?
    5. Berapakah jumlah employee yang diterima setiap bulannya?
    6. Apakah terdapat perbedaan median Salary di tiap Departemen?
    7. Apakah terdapat hubungan Salary dengan Commission yang diperoleh?Jika ada maka bagaimana hubungan kedua variable tersebut?

> # **DATABASE**

## **Connecting To Database**

Bagian ini merupakan langkah awal untuk mulai melakukan proses analisis data. Pertama adalah membuat koneksi ke database di mana seperti yang sudah dijelaskan sebelumnya, database yang akan digunakan adalah database ``HR Database``. Dengan melakukan koneksi ini, kita dapat mengakses seluruh tabel yang ada pada database.

In [1]:
# Import Modules

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import mysql.connector as sqlcon

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# Connect To Database

mydb = sqlcon.connect(
    host = 'localhost',
    user = 'root',
    passwd = 'bbcu010320',
    database = 'humanResource'
)

In [ ]:
# Query Function

curs = mydb.cursor()

def sql_table (query) :
    curs.execute (query)
    result = curs.fetchall()
    df1 = pd.DataFrame(result, columns=curs.column_names)
    return df1

## **Data Detail**

Data pertama ini merupakan data utama yang nantinya akan dianalisa lebih lanjut. Data ini merupakan gabungan dari 6 tabel. Masing-masing dari setiap tabel tersebut diambil beberapa kolomnya dan tidak diambil secara keseluruhan. Informasi-informasi yang dianggap penting saja lah yang diambil. Berikut tabel yang digunakan adalah:

- ```Employees```
    - First_Name
    - Last_Name
    - Jb_ID
    - Salary
    - Manager_ID
    - Department_ID
- ```Departments```
    - Department_Name
- ```Job```
    - Job_Title
- locations
    - State_Province
- ```Countries```
    - Country_ID
    - Country_Name
    - Region_ID

Selain dari tabel, terdapat sebuah kolom bernama Work_Period yang merupakan hasil pengurangan antara Start_Date dan End_Date. 

Semua informasi tersebut kemudian dijadikan dalam sebuah DataFrame yang nantinya akan diolah informasinya.

In [ ]:
# Query 1

df1 = sql_table (
    '''
SELECT CONCAT(E.first_name, ' ', E.last_name) AS Employee_Name, E.Hire_Date, E.Salary, E.Commission_PCT, 
       D.Department_Name, J.Job_Title, L.State_Province, C.Country_Name
    FROM (employees E)
LEFT JOIN departments D ON E.department_id = D.department_id
LEFT JOIN jobs J ON E.Job_ID = J.Job_ID
LEFT JOIN locations L ON d.Location_ID = L.Location_ID
LEFT JOIN countries C ON L.country_ID = C.country_ID;
    '''
)
df1.head()

In [ ]:
# Query 2 (Subquery)

df2 = sql_table(
    '''
SELECT Employee_Id, First_Name, Last_Name, Salary,  
    CASE WHEN salary >= (SELECT AVG(salary) 
FROM employees) 
    THEN 'HIGH'  
    ELSE 'LOW'  
    END AS Salary_Status 
FROM employees;
    '''
)

df2

In [ ]:
# Query 2 (Group By and Agg)
df3 = sql_table(
    '''
SELECT d.department_name,
       e.*
FROM departments d
JOIN
  (SELECT department_id,
  COUNT(employee_id) AS Total_Employee, 
   AVG(salary) AS Avg_Salary, 
   COUNT(commission_pct) AS Total_Commission
FROM employees
GROUP BY department_id) e USING (department_id);
    '''
)

df3

In [ ]:
# Query 3 (CTE / Window Function)

# **DATA MANIPULATION**

Seperti yang telah dijelaskan sebelumnya, data yang digunakan untuk dianalisis adalah data pada ```df1```. Sebelum melakukan analisis lebih lanjut, hal yang harus dilakukan adalah mengecek informasi serta anomali pada data. Jika memang terdapat hal-hal yang dianggap 'kotor' pada data, maka yang perlu dilakukan adalah melakukan penanganan pada bagian tersebut. Pada bagian ini, data akan 'dibersihkan', sehingga output akhir yang diharapkan adalah terdapat sebuah dataset yang bersih yang dapat dianalisis lebih lanjut dengan menampilkan visualisasi, serta melihat statistics-nya.

## **Data Anomalies**

In [ ]:
# Check Info Tabel1
df1.info()

In [ ]:
# Check Missing Value Percentage

df1.isnull().sum()

General Info:
- Terdapat 107 baris dan 8 columns
- Non-null Count menunjukkan bahwa pada columns secara keseluruhan columns tidak memiliki data yang hilang (missing values), namun pada columns Department_Name, State_Province dan Country_Name ditemukan missing values yang masing- masing kehilangan sebanyak 1 dan 35 values.
- Dtype menunjukkan bahwa keseluruhan data memiliki type data yaitu object



## **Handling Anomalies**

In [ ]:
hr = df1.copy()

In [ ]:
hr[hr.isnull().any(axis=1)].head()

# Missing Value in "Country_Name"
- Missing Value in Country_Name shows that missing value has "State_Province" is Oxford
- Based on https://en.wikipedia.org/wiki/Oxford, The United Kingdom has four states: England, Scotland, Wales and Northern Ireland. Oxford is in the county of Oxfordshire, in the state of England. So, we can assume that Oxford is part of England (United Kingdom) and could fill in column 'Country_Name' with 'United Kingdom'

In [ ]:
# Remove Missing Value 1
hr["Country_Name"].fillna("United Kingdom", inplace = True)

In [ ]:
hr[hr.isnull().any(axis=1)].head()

- The table shows that Kimberly Grant has job title as "Sales Representative", so we can assume that "Department name" with kind of job is "Sales"
- Because of the previous explanation, for the column "State_Province" we could fill it with "Oxford"

In [ ]:
# Remove Missing Value 2

hr['Department_Name'].fillna('Sales', inplace = True)
hr['State_Province'].fillna('Oxford', inplace = True)

In [ ]:
# Recheck Info

hr.isnull().sum()

## **Mengubah Tipe Data Yang Salah**

In [ ]:
hr['Hire_Date'] = pd.to_datetime(df1['Hire_Date'])
hr['Salary'] = pd.to_numeric(df1['Salary'])
hr['Commission_PCT'] = pd.to_numeric(df1['Commission_PCT'])


In [ ]:
# Recheck Info
hr.info()

## **Data Duplicate**

Anomali berikutnya yang bisa ditemui adalah data yang duplikat. Tentu saja data yang bersifat duplikat ini akan menjadi sesuatu hal yang akan mengganggu proses analisis data. Jika memang nantinya terdapat data yang duplikat, sebaiknya data duplikatnya dihapus dan disisakan data yang unique saja. Untuk data saat ini, melihat output di atas artinya tidak terdapat data yang duplikat. Dengan begitu tidak perlu ada action yang dilakukan.

In [ ]:
# Check Dupliacate
hr[hr.duplicated()]

### General Info Data Cleaned

In [ ]:
listItem = []
for col in hr.columns :
    listItem.append([col, hr[col].dtype, len(hr),hr[col].isna().sum(), round((hr[col].isna().sum()/len(hr[col])) * 100,2),
                    hr[col].nunique(), list(hr[col].drop_duplicates().values)])

hrNew = pd.DataFrame(columns=['Column Name', 'Data Type', 'Data Count', 'Missing Value', 
    'Missing Value Percentage', 'Number of Unique', 'Unique Sample'],
                     data=listItem)

hrNew

## **Data Outlier**

In [ ]:
# Outlier Check With Function

Q1_amount = hr['Salary'].describe()['25%']
Q3_amount = hr['Salary'].describe()['75%']
iqr = Q3_amount - Q1_amount

outlier_index = hr[(hr['Salary'] < Q1_amount - (1.5 * iqr)) | (df1['Salary']> Q3_amount + (1.5 * iqr)) ].index
not_outlier_index = hr[(hr['Salary'] > Q1_amount - (1.5 * iqr)) & (df1['Salary']< Q3_amount + (1.5 * iqr)) ].index
hr.loc[outlier_index]

In [ ]:
plt.style.use('seaborn')
plt.figure(figsize=(8,5))
sns.boxplot(x=hr['Salary'])
plt.show()

In [ ]:
plt.style.use('seaborn')
plt.figure(figsize=(8,5))
sns.boxplot(x=hr['Commission_PCT'])
plt.show()

- Boxplot is a good statistical graphic to analyze the dataset and indentify outliers values. An outlier is as observation that lies an abnormal distance from other values, in this case the analyst have to decide what is considered abnormal.

- The boxplots below, give the information about the data distributions:
    - Salary and Commission_PCT has a skewed right(positive)ditribution.

#### Handling Outliers
- Take a look at the column for employees' "Salary". So, it's clearly visible 24000.0 is not in the range and it doesn't fall in between 2100k$ to 8900$. So, It indicates an outlier of this salary column.
- We could just drop the outliers, but I want to know the overall salary without looking at the Job_Title. So, we can assume that "President" as ordinary Employee

## Add New Feature

### Salary Status
- Create a new column to categorize employee salary levels. This column is intended to assist in further analysis. I desire to see if Salary status is affected by each employee's department. In case yes, then how big is the correlation?

In [ ]:
hr['Salary_Status'] = np.where(hr['Salary'] < hr['Salary'].mean(), 'Low','High')
hr.head()

### Month Feature

- Column "month" purpose to see how many employees are hired.

In [ ]:
hr['Month'] = pd.to_datetime(hr['Hire_Date']).dt.strftime('%B')

# **DATA VISUALIZATION & STATISTICS**

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly
from plotly import version
print (version)
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [ ]:
hr.columns

### Proportion of employees in each Department

In [ ]:
hr_Dept = hr.groupby(['Department_Name']).count()['Employee_Name'].reset_index()
hr_Dept

In [ ]:
fig = px.pie(hr_Dept,
             names = 'Department_Name', 
             values= 'Employee_Name',
             title = 'Countries by Employees')
fig.show()

- Based on diagram pie, we can see that the dominance of employees in the Shipping Department and the Public Relations Department has the least number of employees
- It can be assumed that the company in question is a company concerned in delivery service

### TOP 5 Employee with High  Salary

In [ ]:
sns.barplot(
    x='Salary',
    y='Job_Title',
    hue='Department_Name',
    data=hr.nlargest(5, 'Salary'),
    palette='RdYlBu'
)

- Behind every successful company, there are the effort worthy employee working. Right from lower to Hight-level management. These 5 of those employees who earns the highest salary
    1. President
    2. Administration Vice President (2 Employee)
    3. Sales MAnager (2 Employee)

### Salary vs Department

In [ ]:
sns.catplot(x = 'Salary_Status', hue='Department_Name', kind ='count', height = 5, aspect=2.5, data=hr, palette='tab20c')
plt.title("Salary Status per Department", fontsize=25)

- In the graphic Salaries by department is possible to see the distribuition of the salaries by department.

    - Most of the employees of the sales department have low or high salaries
    - Purchasing department is in the second place where most of the employees receives low and high salaries.

### Commission vs Job_Title

In [ ]:
hr.groupby('Department_Name')['Commission_PCT'].value_counts()

In [ ]:
sns.catplot(x = 'Commission_PCT', hue='Job_Title', kind ='count', height = 7,aspect=2.5, data=hr, palette='Set2' )
plt.title("Commission by Job_Title", fontsize=14)

- Diagram diatas menunjukkan bahwa yang mendapatkan Commission hanya departemen Sales

### Employee vs Country

In [ ]:
hr_country = hr.groupby(['Country_Name']).count()['Department_Name'].reset_index()
hr_country

In [ ]:
fig = px.pie(hr_country,
             names = 'Country_Name', 
             values= 'Department_Name',
             title = 'Countries by Employees')
fig.show()

- Dominasi Employee pada perusahaan berasal dari United State of America dan yang paling sedikit yaitu berasal dari negara Germany

### Jumlah Employee yang di hired tiap bulan

In [ ]:
hr.sort_values(by='Month', ascending=True).groupby('Department_Name')['Month'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(hr['Month'],
            order = hr['Month'].value_counts().index)
plt.xlabel('Month', size = 20)
plt.ylabel('Total Employee', size = 20)
plt.title ('Hired Employee per Month', size = 30)
plt.show()

## Statistics

### Perbedaan Salary tiap Department

In [ ]:
# Uji Normalitas 'Salary' yang diperoleh
from scipy.stats import shapiro

shapiro(hr['Salary'])

sw_statistic, sw_pvalue = shapiro(hr['Salary'])

if sw_pvalue > 0.05:
    print(f'Terima H0 Karena P-Value: {sw_pvalue} > 5%. So, we assume a normal distribution')
else:
    print(f'Tolak H0 Karena P-Value {sw_pvalue} < 5%. So, we do not assume a normal distribution')

In [ ]:
# Uji Perbandingan Salary yang diperoleh berdasarkan Departement
# untuk menguji perbandingan lebih dari 2 sampel independent dalam statistik non parametrik

from scipy.stats import kruskal

krus, pvalkrus = kruskal(
    hr[hr['Salary'] == 'Shipping']['Salary'],
    hr[hr['Salary'] == 'Finance']['Salary'],
    hr[hr['Salary'] == 'Purchasing']['Salary'],
    hr[hr['Salary'] == 'IT']['Salary'],
    hr[hr['Salary'] == 'Executive']['Salary'],
    hr[hr['Salary'] == 'Marketing']['Salary'],
    hr[hr['Salary'] == 'Accounting']['Salary'],
    hr[hr['Salary'] == 'Administration']['Salary'],
    hr[hr['Salary'] == 'Human Resources']['Salary'],
    hr[hr['Salary'] == 'Public Relations']['Salary']
) 

if pvalkrus > 0.05 :
    print (f'Accept H0 Karena P-Value ({pvalkrus} > 5%)')
    print ('Tidak Terdapat Perbedaan Nilai Median Salary pada Setiap Departement')
else :
    print (f'Reject H0 Karena P-Value ({pvalkrus} < 5%)')
    print ('Terdapat Perbedaan Nilai Median Salary pada Setiap Departement')

### Hubungan antara Salary dengan Commission

In [ ]:
# First, check the correlation between two variable to decide which one method (spearman/pearson) we have to use

sns.scatterplot(x = hr['Salary'], y = hr['Commission_PCT'])
plt.show()

In [ ]:
hr[['Salary', 'Commission_PCT']].corr(method='spearman')

- Dari output di atas, diperoleh hasil bahwa kedua data tidak berdistribusi normal pada taraf signifikansi 5%. Hal ini dikarenakan p-value kedua data < 0.05. Karena kedua data tidak berdisribusi normal, maka akan digunakan uji korelasi spearman untuk mengetahui ada tidak nya korelasi antar kedua variabel.
- Hasilnya 0.5 Positif Moderat, artinya apabila nilai "Salary" meningkat, nilai "Commission_PCT" juga meningkat